In [26]:
import pandas as pd
import pyarrow.parquet as pq
import glob
from tqdm import tqdm
tqdm.pandas()

In [27]:
def dataframe(path, cols):
	paths = glob.glob(path)
	dataset = pq.ParquetDataset(paths)
	table = dataset.read()
	df = table.to_pandas()
	df = df[cols]
	return df

In [28]:
df = dataframe('base/*/*.parquet', ['word', 'definition', 'score'])

In [29]:
df['spaces'] = df.progress_apply(lambda row: ' ' in row.word, axis=1)

100%|██████████| 1284585/1284585 [00:33<00:00, 38862.77it/s]


In [30]:
df['se'] = df.progress_apply(lambda row: row.word.lower() in row.definition.lower(), axis=1)

100%|██████████| 1284585/1284585 [01:03<00:00, 20320.36it/s]


In [31]:
df = df[df['se'] == False][df['spaces'] == False]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [32]:
df['pat'] = df['score'].apply(lambda s: round(s, 1))

In [34]:
df = df.drop(['spaces', 'se'], axis=1)

In [36]:
df.to_parquet('output', engine='pyarrow', partition_cols = ['pat'])